In [162]:
import fastf1
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import General

In [163]:
os.makedirs('cache', exist_ok=True)
fastf1.Cache.enable_cache('cache')

In [ ]:
def GetLapTimesForDriver(year, raceName, session):
    sess = fastf1.get_session(year, raceName, session)
    sess.load()
    LapTimes = []
    max_laps = 0
    for d in sess.drivers:
        laps = sess.laps.pick_driver(d)
        lap_times = []
        lt = laps["LapTime"]
        for item in lt:
            x = item.total_seconds()
            if np.isnan(x):
                x = 0
            lap_times.append(x)

        LapTimes.append(lap_times)
    df = pd.DataFrame(LapTimes, index=sess.drivers)
    df = df.T

    


In [165]:
round = 1
year = General.GetAllYears()[0]
race= General.GetAllRaces(year)[round]
df = GetLapTimesForDriver(year,race,"R")
sess = fastf1.get_session(2025, race, "R")
sess.load()


core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '2

In [166]:
finalres = sess.results["DriverNumber"]
winner = finalres[0]

C:\Users\Bibidh Subedi\AppData\Local\Temp\ipykernel_14212\683315012.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  winner = finalres[0]


In [167]:
dff = df.replace(np.nan, 0)
def classify_driver(column):
    if dff[column].dropna().eq(0).all():
        return "DNS"
    if dff[column].notna().any() and (dff[column].iloc[-1] == 0 or dff[column].isna().any()):
        return "DNF"
    return "Finished"

driver_status = {col: classify_driver(col) for col in dff.columns}
driver_status
print(driver_status)

fin = []
not_fin = []
for d in driver_status:
    if driver_status[d]=="Finished":
        fin.append(d)
    else:
        not_fin.append(d)

{'4': 'Finished', '1': 'Finished', '63': 'Finished', '12': 'Finished', '23': 'Finished', '18': 'Finished', '27': 'Finished', '16': 'Finished', '81': 'Finished', '44': 'Finished', '10': 'Finished', '22': 'Finished', '31': 'Finished', '87': 'Finished', '30': 'DNF', '5': 'DNF', '14': 'DNF', '55': 'DNS', '7': 'DNS', '6': 'DNS'}


In [168]:
df = df[fin]
df = df.replace(0, np.nan)
df = df.apply(lambda col: col.fillna(col.mean()), axis=0)
df.head()

,4,1,63,12,23,18,27,16,81,44,10,22,31,87
0,117.099000,119.392000,124.64400,142.35200,132.195000,139.48500,141.221000,128.480000,120.807000,132.883000,134.132000,130.655000,145.769000,149.413000
1,103.428302,103.341151,103.68634,104.57937,104.672389,104.73013,104.740167,103.933528,102.084462,103.967189,104.889167,104.085698,103.848698,103.870075
2,103.428302,103.341151,103.68634,104.57937,104.672389,104.73013,104.740167,103.933528,102.084462,103.967189,104.889167,104.085698,103.848698,103.870075
3,103.428302,103.341151,103.68634,104.57937,104.672389,104.73013,104.740167,103.933528,102.084462,103.967189,104.889167,104.085698,103.848698,103.870075
4,143.648000,143.609000,141.71200,139.16300,142.084000,140.40900,139.058000,142.597000,143.155000,141.772000,141.360000,141.441000,130.528000,125.412000


In [ ]:
import pandas as pd

winner = df.columns[0]
winner_cum = df[winner].cumsum()
delta_to_winner = df.cumsum().subtract(winner_cum, axis=0)
sum_delta = delta_to_winner.sum(skipna=True)

worst = sum_delta.max()
dnf_value = 1.5 * worst
n = len(not_fin)
dnf_series = pd.Series([dnf_value]*n, index=not_fin)

sum_delta_final = pd.concat([sum_delta, dnf_series])

print(sum_delta_final)


4        0.000000
1      216.606925
63     911.138019
12    2954.388960
23    2727.724016
18    3109.118238
27    3324.711349
16    1802.767830
81    -791.367843
44    2122.492057
10    2853.362349
22    2126.456717
31    2945.444396
87    3393.497962
30    5090.246943
5     5090.246943
14    5090.246943
55    5090.246943
7     5090.246943
6     5090.246943
dtype: float64
